In [1]:
from dataclasses import dataclass, field
@dataclass
class Trace:
    """Entity time series (xs and ys) with meta data (name, unit,...)
    """
    entity: str
    unit: str
    name: str = ""
    is_cumsum: bool = False
    is_diff: bool = False
    xs: list = field(default_factory=list, compare=False, hash=False, repr=False)
    ys: list = field(default_factory=list, compare=False, hash=False, repr=False)
    
    def get_label(self) -> str:
        """Returns a trace label like 'garden temperature (°C)'
        """
        label = self.name if self.name else self.entity
        
        if self.is_cumsum:
            label += ".cumsum"
        if self.is_diff:
            label += ".diff"

        return label + f" ({self.unit})"
        

In [2]:
trace = Trace(entity = "shelly_uni2_temperature", unit = "°C", xs=[1,2,3], ys=[7,3,4])
trace

Trace(entity='shelly_uni2_temperature', unit='°C', name='', is_cumsum=False, is_diff=False)

In [3]:
import json
json.dumps(trace)

TypeError: Object of type Trace is not JSON serializable

In [4]:
import pandas as pd
import datetime as dt 
import matplotlib.pyplot as plt
import plotly
import plotly.express as px
import plotly.graph_objs as go  

In [5]:
dict(mode="lines")

{'mode': 'lines'}

In [6]:
fig = go.Figure()
scatter_kwargs = dict(mode="lines")  # scatter defaults
fig.add_trace(go.Scatter(x=trace.xs, y=trace.ys, name=trace.entity, **scatter_kwargs))

In [7]:
# Using plotly.express
import plotly.express as px

df = px.data.stocks()
fig = px.line(df, x='date', y="GOOG")
fig.show()

In [8]:
df.head()

,date,GOOG,AAPL,AMZN,FB,NFLX,MSFT
0,2018-01-01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,2018-01-08,1.018172,1.011943,1.061881,0.959968,1.053526,1.015988
2,2018-01-15,1.032008,1.019771,1.053240,0.970243,1.049860,1.020524
3,2018-01-22,1.066783,0.980057,1.140676,1.016858,1.307681,1.066561
4,2018-01-29,1.008773,0.917143,1.163374,1.018357,1.273537,1.040708


In [9]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/finance-charts-apple.csv')

fig = px.line(df, x='Date', y='AAPL.High', title='Time Series with Rangeslider')

fig.update_xaxes(rangeslider_visible=True)
fig.show()

In [10]:
import plotly.graph_objects as go

# Create random data with numpy
import numpy as np
np.random.seed(1)

N = 100
random_x = np.linspace(0, 1, N)
random_y0 = np.random.randn(N) + 5
random_y1 = np.random.randn(N)
random_y2 = np.random.randn(N) - 5

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=random_x, y=random_y0,
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=random_x, y=random_y1,
                    mode='lines+markers',
                    name='lines+markers'))
fig.add_trace(go.Scatter(x=random_x, y=random_y2,
                    mode='markers', name='markers'))
fig.update_layout(yaxis_title='Temperature (degrees F)')
fig.show()

In [18]:
dt.datetime.now().date() - dt.timedelta(365)

datetime.date(2023, 5, 4)

In [17]:
dt.datetime(sd.year, sd.month-1, sd.day).date()

datetime.date(2024, 4, 3)